In [1]:
from scipy.integrate import ode
from scipy.interpolate import interp1d
import pandas as pd
from astropy import units as u
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from scipy.optimize import minimize, approx_fprime
from joblib import Parallel, delayed

Capture hides output

In [2]:
%%capture
# %run cowling_approximation2.ipynb
# %run cowling_approximation2_invert_dedp.ipynb
# %run cowling_approximation.ipynb
# %run cowling_approximation2_invert_dedp_extrapolate.ipynb
# %run cowling_approximation3.ipynb
%run cowling_approximation3_wu_equations.ipynb

In [3]:
def process(k):
    df = pd.read_csv(path)
    e_old, p_old = df_to_ep(df)

    ### Polytrope ###
#     p = p_old
#     e = e_func(p)

    ### EOS DATA ###
    p = p_old
    e = e_old


    EOS = get_ep(e, p)
    
    
    # Initial Conditions
    km2cm = 1e5
    r_i = 1
    p0 = p[k]
    e0 = EOS(p0)
    p_c = p0 - 2 * np.pi * (G/(c**4)) * r_i**2 * (p0 + e0) * (3*p0 + e0)/3
    e_c = EOS(p_c)
    m0 = e_c/(c**2) * 4/3 * np.pi * r_i**3


    omega = 2e3*(2*np.pi) #Arbitrary guess
    l=2

    v0 = -1 # Two step integration optimization: similar to phi previously.

    W0 = 1
    U0 = W0/(l * np.exp(v0))


    init_VEC = [p_c, m0, v0, W0, U0]
#     p_min = min(p) 
    
#     p_smooth, dedp = dedp_arr(p, EOS)
#     dedp_EQ = get_dedp(dedp, p_smooth)
    dedp_EQ = None
    
    # Integrate
    p, m, r_arr, v, w, u = tov(EOS, init_VEC, r_i, p_min, 
                                 omega, dedp_EQ,
                               l = l)

    max_idx = np.argmax(m)
    m_R = m.max() # In units of msun
    r_R = r_arr[max_idx] # In units of km
    p_R = p[max_idx] #cgs
    ec_R = EOS(p_R) #cgs
    u_R = u[max_idx] #cgs
    v_R = v[max_idx]
    w_R = w[max_idx]

    schild = (1-2*G*m_R/(c**2*r_R))
    interior = np.exp(v_R)

    # Computer new v0, W0, U0
    v_ext = np.log(schild) #At surface
    v_int = v_R #At surface
    delta_v = v_int - v_ext
    v0 =  v0 - delta_v

    W0 = 1
    U0 = W0/(l * np.exp(v0))
    init_VEC = [p_c, m0, v0, W0, U0]

    p, m, r_arr, v, w, u = tov(EOS, init_VEC, r_i, p_min, omega, 
                               dedp_EQ, l = l)

    max_idx = np.argmax(m)
    m_R = m.max() # In units of msun
    r_R = r_arr[max_idx] # In units of km
    p_R = p[max_idx] #cgs
    ec_R = EOS(p_R) #cgs
    u_R = u[max_idx] #cgs
    v_R = v[max_idx]
    w_R = w[max_idx]

    schild = (1-2*G*m_R/(c**2*r_R))
    interior = np.exp(v_R)
    
    def minimize_boundary_mass_plot(params, p = p, EOS = EOS):
        # Repeat integration

        omega = params

        # Integrate
        p, m, r_arr, v, w, u = tov(EOS, init_VEC, r_i, p_min, omega,dedp_EQ, l = l)

        max_idx = np.argmax(m)
        m_R = m.max() # In units of msun
        r_R = r_arr[max_idx] # In units of km
        p_R = p[max_idx] #cgs
        ec_R = EOS(p_R) #cgs
        u_R = u[max_idx] #cgs
        v_R = v[max_idx]
        w_R = w[max_idx] 

        loss = np.log10(abs(boundary_wu(r_R, m_R, omega, w_R, u_R)))
        return loss

    omega_guess = 2e3*(2*np.pi)
    init_guess = [omega_guess]
    res = minimize(minimize_boundary_mass_plot, x0 = init_guess, 
                   method='Nelder-Mead', 
                   options = {"disp": False, "maxiter":15},
                   tol = 1)

    omg = res.x[0]
    f = omg/(2*np.pi)
    return f, m_R, r_R, k

In [4]:
# def process(k):
#     df = pd.read_csv(path)
#     e, p = df_to_ep(df)
#     EOS = get_ep(e, p)

#     # Initial Conditions
#     km2cm = 1e5
#     r_i = 1
#     p0 = p[k]
#     e0 = EOS(p0)
#     p_c = p0 - 2 * np.pi * (G/(c**4)) * r_i**2 * (p0 + e0) * (3*p0 + e0)/3
#     e_c = e0
#     m0 = e_c/(c**2) * 4/3 * np.pi * r_i**3

#     phi0 = -1 #Arbitrary guess 
#     omega = 2e3*(2*np.pi) #Arbitrary guess

#     C = 1e-16 # Arbitrary constant
#     l = 2
#     W0 =  C   * (r_i ** (l+1))
#     V0 = -C/l * (r_i ** (l))

#     init_VEC = [p_c, m0, phi0, W0, V0]
#     p_min = min(p) # set a more conservative minimum pressure maybe? 

#     # Integrate
#     p, m, r_arr, phi, w, v = tov(EOS, init_VEC, r_i, p_min, omega,progress = True, 
#                                  l = l, n_iter_max = 2000)

#     max_idx = np.argmax(m)
#     m_R = m.max() # In units of msun
#     r_R = r_arr[max_idx] # In units of km
#     p_R = p[max_idx] #cgs
#     ec_R = EOS(p_R) #cgs
#     phi_R = phi[max_idx] #cgs
#     v_R = v[max_idx]
#     w_R = w[max_idx]

#     schild = (1-2*G*m_R/(c**2*r_R))
#     interior = np.exp(v_R)

#     # Computer new phi0
#     delta_phi = phi_int - phi_ext
#     phi0 =  phi0 - delta_phi

#     W0 =  C   * (r_i ** (l+1))
#     V0 = -C/l * (r_i ** (l))

#     init_VEC = [p_c, m0, phi0, W0, V0]
#     time.sleep(0.2)

#     # Integrate
#     p, m, r_arr, phi, w, v = tov(EOS, init_VEC, r_i, p_min, omega, progress = True, l = l)

#     max_idx = np.argmax(m)
#     m_R = m.max() # In units of msun
#     r_R = r_arr[max_idx] # In units of km
#     p_R = p[max_idx] #cgs
#     ec_R = EOS(p_R) #cgs
#     phi_R = phi[max_idx] #cgs
#     v_R = v[max_idx]
#     w_R = w[max_idx]

#     schild = (1-2*G*m_R/(c**2*r_R))
#     interior = np.exp(v_R)
    
#     def minimize_boundary_mass_plot(params, p = p, EOS = EOS):
#         # Repeat integration

#         omega = params

#         # Integrate
#         p, m, r_arr, phi, w, v = tov(EOS, init_VEC, r_i, p_min, omega, progress = False, 
#                                  l = l)
#         max_idx = np.argmax(m)
#         m_R = m.max() # In units of msun
#         r_R = r_arr[max_idx] # In units of km
#         p_R = p[max_idx] #cgs
#         ec_R = EOS(p_R) #cgs
#         phi_R = phi[max_idx] #cgs
#         v_R = v[max_idx]
#         w_R = w[max_idx]

#         loss = np.log10(abs(boundary_wv(r_R, p_R, m_R, omega, phi_int, v_R)))
#         return loss
    
#     omega_guess = 2e3*(2*np.pi)
#     init_guess = [omega_guess]
#     res = minimize(minimize_boundary_mass_plot, x0 = init_guess, 
#                    method='Nelder-Mead', 
#                    options = {"disp": False, "maxiter":15},
#                    tol = 1)

#     omg = res.x[0]
#     f = omg/(2*np.pi)
#     return f, m_R, r_R

In [ ]:
f_mode_list = []
mass_list = []
radius_list = []

if path == "eos_data/nl3cr_short.csv":
    print("EOS: NL3CR (Short)")
    ind_start = -3
    ind_stop = -1180
    jump = 20 # Default should be 1 for small EOS data
    
if path == "eos_data/nl3cr.csv" :
    print("EOS: NL3CR (Full)")
    ind_start = -2
    ind_stop = -1140
    jump = 20 # Default should be 1 for small EOS data
    
if (path == "eos_data/sly230a_short.csv") \
    or (path =="eos_data/sly230a.csv"):
    print("EOS: SLY230A (Full)")
    ind_start = -2
    ind_stop = -65
    jump = 1 # Default should be 1 for small EOS data
    
if path == "eos_data/sly_short.csv":
    print("EOS: SLY4 (Short)")
    ind_start = -2
    ind_stop = -28
    jump = 1 # Default should be 1 for small EOS data
    
if path == "eos_data/sly.csv": 
    print("EOS: SLY4 (Full)")
    ind_start = -2
    ind_stop = -35
    jump = 1 # Default should be 1 for small EOS data
    
vals = range(ind_stop, ind_start+1, 1)[::-jump]

results = Parallel(n_jobs=-2, 
                   verbose=0, 
                   max_nbytes='8M')(delayed(process)(k) for k in tqdm(vals))

EOS: NL3CR (Full)


In [ ]:
mass_arr = np.array(results).T[1]
f_mode_arr = np.array(np.array(results).T[0])
radius_arr = np.array(results).T[2]
idx_arr = np.array(results).T[3]
hz2khz = 1e-3

In [ ]:
max_idx = mass_arr.argmax()

In [ ]:
idx_arr[(np.abs(mass_arr/msun - 1.4)).argmin()]

In [ ]:
mass_arr = mass_arr[max_idx:]
f_mode_arr = f_mode_arr[max_idx:]
radius_arr = radius_arr[max_idx:]

In [ ]:
max_idx_new = mass_arr.argmax()

print(f"M_max = {mass_arr[max_idx_new]/msun}")
print(f"R_max = {radius_arr[max_idx_new]/km2cm}")
print(f"f_max = {f_mode_arr[max_idx_new]}")

print()
solar_idx = (np.abs(mass_arr/msun - 1.4)).argmin()
print(f"M_1.4 = {mass_arr[solar_idx]/msun}")
print(f"R_1.4 = {radius_arr[solar_idx]/km2cm}")
print(f"f_1.4 = {f_mode_arr[solar_idx]}")

In [ ]:
# mass_arr = np.array(mass_list)
# f_mode_arr = np.array(f_mode_list)
# radius_arr = np.array(radius_list)
# hz2khz = 1e-3

plt.figure(dpi = 300)
plt.tight_layout()
plt.scatter(mass_arr/msun, f_mode_arr*hz2khz, c = radius_arr/km2cm, marker = "x", 
            cmap = "plasma")
plt.xlabel("Mass/Msun")
plt.ylabel("fmode (kHz)")
cbar = plt.colorbar()
cbar.set_label('Radius (km)', rotation=-90, labelpad=15)
utkarshGrid()
plt.xlim(1, 2.5)
plt.ylim(1.5, 3.5)
plt.show()

In [ ]:
plt.plot(np.sqrt((mass_arr/msun)/((radius_arr/km2cm)**3)), f_mode_arr*hz2khz)
plt.xlabel("√M/R^3")
plt.ylabel("fmode (kHz)")
plt.xlim(0.02, 0.05)
plt.ylim(1.4, 3)
plt.show()

In [ ]:
# plt.plot(np.sqrt((mass_arr/(1.4*msun))/((radius_arr/(10*km2cm))**3)), f_mode_arr*hz2khz)
# plt.ylabel("f(khz)")
# plt.show()

In [ ]:
plt.plot(radius_arr/km2cm, mass_arr/msun)
plt.xlabel("Radius (km)")
plt.ylabel("Mass (Msun)")
plt.show()

In [ ]:
# np.savetxt("mr_sly.csv", np.array([radius_arr, mass_arr]).T)